In [1]:
# import biopandas
from biopandas.pdb import PandasPdb
from biopandas.mol2 import PandasMol2
import numpy as np
import scipy.spatial as sp
import pandas as pd

In [2]:
# read protein pdb file

pdb_id = '1a1e'
protein = PandasPdb().read_pdb(pdb_id + '/' + pdb_id + '_protein.pdb')
#protein.df['ATOM'].iloc[[1639]]


In [3]:
#getting the number of chains and chain_id

chain = np.unique(protein.df['ATOM']['chain_id'])
chain

array(['A', 'B'], dtype=object)

In [4]:
# reading the ligand file

ligand = PandasMol2().read_mol2(pdb_id + '/' + pdb_id + '_ligand.mol2')
ligand_noH = ligand.df.loc[ligand.df['atom_type'] != 'H']
ligand_coord = ligand_noH[["x", "y", "z"]].values

In [5]:
# getting the residue numbers of each chain those are within the cutoff distance of the ligand heavy atoms

protein_pocket = pd.DataFrame()

for ch in chain:
    protein_chain = protein.df['ATOM'].loc[protein.df['ATOM']['chain_id'] == ch].reset_index()
    chain_coord = protein_chain[["x_coord","y_coord","z_coord"]].values
    distance_protein = sp.distance_matrix(ligand_coord, chain_coord, p=2)
    indexarr_chain= np.unique(np.argwhere(distance_protein < 8)[:,1])
    res_id = np.unique(protein_chain.loc[protein_chain.index.isin(indexarr_chain)]['residue_number'])
    protein_seg = protein_chain.loc[protein_chain['residue_number'].isin(res_id)]
    protein_pocket = pd.concat([protein_pocket, protein_seg])

protein.df['ATOM'] = protein_pocket.drop(['index'], axis=1).reset_index().drop(['index'], axis=1)
protein.df['ATOM']


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1782,,N,,LYS,,B,155,,...,36.647,6.515,30.602,1.0,35.68,,,N,NaN,1801
1,ATOM,1783,,H,,LYS,,B,155,,...,36.193,7.299,30.090,1.0,0.00,,,H,NaN,1802
2,ATOM,1784,,CA,,LYS,,B,155,,...,36.879,5.269,29.919,1.0,36.15,,,C,NaN,1803
3,ATOM,1785,,HA,,LYS,,B,155,,...,36.868,4.481,30.672,1.0,0.00,,,H,NaN,1804
4,ATOM,1786,,C,,LYS,,B,155,,...,38.252,5.067,29.287,1.0,36.67,,,C,NaN,1805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,ATOM,3102,,CB,,CYS,,B,241,,...,48.772,-2.218,47.321,1.0,27.85,,,C,NaN,3121
604,ATOM,3103,,HB1,,CYS,,B,241,,...,48.451,-3.157,46.871,1.0,0.00,,,H,NaN,3122
605,ATOM,3104,,HB2,,CYS,,B,241,,...,49.857,-2.219,47.426,1.0,0.00,,,H,NaN,3123
606,ATOM,3105,,SG,,CYS,,B,241,,...,48.292,-0.852,46.229,1.0,35.25,,,S,NaN,3124


In [6]:
# getting the hetatms within the ligand cutoff distance

hetatm_coord = protein.df['HETATM'][["x_coord","y_coord","z_coord"]].values
distance_hetatm = sp.distance_matrix(ligand_coord, hetatm_coord, p=2)
indexarr_hetatm = np.unique(np.argwhere(distance_hetatm < 8)[:,1])
protein.df['HETATM'] = protein.df['HETATM'].loc[protein.df['HETATM'].index.isin(indexarr_hetatm)]
indexarr_hetatm

array([26, 27, 30, 31, 39, 42, 43])

In [7]:
# writing the pdb file that for the pocket

protein.to_pdb(path='protein_stripped.pdb', 
            records=None, 
            gz=False, 
            append_newline=True)

In [8]:
# cross check: residue ids in the calculated pocket structure

pocket_res_calculated = np.unique(protein.df['ATOM']['residue_number'])
pocket_res_calculated

array([155, 156, 157, 158, 159, 161, 162, 178, 179, 180, 181, 182, 183,
       186, 187, 188, 189, 190, 191, 193, 201, 202, 203, 204, 205, 206,
       207, 208, 216, 217, 218, 219, 233, 237, 238, 239, 240, 241])

In [9]:
# residue ids from the given pocket structure in PDBbind

protein_pocket = PandasPdb().read_pdb(pdb_id + '/' + pdb_id + '_pocket.pdb')

pocket_res_file = np.unique(protein_pocket.df['ATOM']['residue_number'])
pocket_res_file

array([155, 156, 157, 158, 159, 161, 162, 178, 179, 180, 181, 182, 183,
       186, 187, 188, 189, 190, 191, 193, 201, 202, 203, 204, 205, 206,
       207, 208, 216, 217, 218, 219, 233, 237, 238, 239, 240, 241])

In [10]:
print(protein.df['ATOM'].shape)
print(protein_pocket.df['ATOM'].shape)

(608, 21)
(386, 21)


In [11]:
# Though the residue number matches, the number of atoms in each structure does not match
# to compare, considered only the heavy atomm
protein_noH = protein.df['ATOM'][protein.df['ATOM']['element_symbol'] != 'H']
print(protein_noH.shape)

protein_pocket_noH = protein_pocket.df['ATOM'][protein_pocket.df['ATOM']['element_symbol'] != 'H']
print(protein_pocket_noH.shape)

(308, 21)
(308, 21)


In [15]:
protein_H = protein.df['ATOM'][protein.df['ATOM']['element_symbol'] == 'H']['atom_name']
np.unique(protein_H)

array(['1HD1', '1HD2', '1HG1', '1HG2', '1HH1', '1HH2', '2HD1', '2HD2',
       '2HG1', '2HG2', '2HH1', '2HH2', '3HD1', '3HD2', '3HG1', '3HG2',
       'H', 'HA', 'HA1', 'HA2', 'HB', 'HB1', 'HB2', 'HB3', 'HD1', 'HD2',
       'HE', 'HE1', 'HE2', 'HG', 'HG1', 'HG2', 'HH', 'HZ1', 'HZ2', 'HZ3'],
      dtype=object)

In [16]:
protein_pocket_H = protein_pocket.df['ATOM'][protein_pocket.df['ATOM']['element_symbol'] == 'H']['atom_name']
np.unique(protein_pocket_H)

array(['1HD2', '1HH1', '1HH2', '2HD2', '2HH1', '2HH2', 'H', 'HE', 'HG',
       'HG1', 'HH', 'HZ1', 'HZ2', 'HZ3'], dtype=object)

In [ ]:
from pathlib import Path

p = Path('PDBbind/refined-set/')
subdirectories = [x for x in p.iterdir() if x.is_dir()]

print(str(subdirectories[0]))

In [ ]:
i = 0

out_path = './stripped_proteins/'
for dir_name in subdirectories:
    name = str(dir_name).split('/')[2]
    protein_file_name = str(dir_name) + '/' + name + '_protein.pdb'
    ligand_file_name = str(dir_name) + '/' + name + '_ligand.mol2'
    #print(ligand_file_name)
    protein = PandasPdb().read_pdb(protein_file_name)
    protein_coord = protein.df['ATOM'][["x_coord","y_coord","z_coord"]].values
    ligand = PandasMol2().read_mol2(ligand_file_name)
    ligand_coord = ligand.df[["x", "y", "z"]].values
    
    distance_matrix = sp.distance_matrix(ligand_coord, protein_coord, p=2, threshold=1000000)
    indexArr = np.argwhere(distance_matrix < 10)[:,1]
    indexArr = np.unique(indexArr)
    protein.df['ATOM'] = protein.df['ATOM'].iloc[indexArr]
    protein.df['HETATM'] = protein.df['HETATM'].loc[protein.df['HETATM']['record_name'] != 'HETATM']
    
    out_file_name = out_path + name + '.pdb'
    
    
    protein.to_pdb(path=out_file_name, 
            records=None, 
            gz=False, 
            append_newline=True)
    
    
    i = i +1
    print(i)


In [ ]:
protein.df['HETATM'] = protein.df['HETATM'].loc[protein.df['HETATM'].index.isin(indexArr_hetatm)]


In [ ]:
protein.to_pdb(path='protein_stripped.pdb', 
            records=None, 
            gz=False, 
            append_newline=True)

In [ ]:
protein.df['ATOM'].loc[protein.df['ATOM']['residue_number'].isin([138])]

In [ ]:
protein_pocket = PandasPdb().read_pdb('1ajn/1ajn_pocket.pdb')

In [ ]:
np.unique(protein_pocket.df['ATOM']['chain_id'])

In [ ]:
pocket_res = np.unique(protein_pocket.df['ATOM']['residue_number'])

In [ ]:
pocket_res1 = np.unique(protein.df['ATOM']['atom_name'])

In [ ]:
pocket_res

In [ ]:
res_id

In [ ]:
protein_pocket.df['ATOM']

In [ ]:
protein.df['ATOM'] = protein.df['ATOM'].loc[protein.df['ATOM']['residue_number'].isin(pocket_res)]

In [ ]:
protein.df['ATOM']=  protein.df['ATOM'][protein.df['ATOM']['line_idx'] > ter_pos[0]]
protein.df['ATOM']

In [ ]:
protein_pocket.df['ATOM'].iloc[:10]

In [ ]:
protein_pocket.df['ATOM'] = protein_pocket.df['ATOM'][protein_pocket.df['ATOM']['element_symbol'] != 'H']

In [ ]:
protein_pocket.df['ATOM']

In [ ]:
protein = PandasPdb().read_pdb('PDBbind/refined-set/1ajn/1ajn_protein.pdb')
#protein.df['ATOM'].iloc[[1639]]
protein.df['ATOM'] = protein.df['ATOM'][protein.df['ATOM']['element_symbol'] != 'H']

alpha_carbon = protein.df['ATOM'][protein.df['ATOM']['atom_name'] == 'CA']

In [ ]:
alpha_carbon

In [ ]:
alpha_carbon = alpha_carbon.reset_index()

In [ ]:
alpha_carbon

In [ ]:
alpha_carbon = alpha_carbon.drop(['index'], axis=1)

In [ ]:
alpha_carbon

In [ ]:
alpha_coord = alpha_carbon[["x_coord","y_coord","z_coord"]].values

In [ ]:
ligand = PandasMol2().read_mol2('PDBbind/refined-set/1ajn/1ajn_ligand.mol2')
ligand_noH = ligand.df[ligand.df['atom_type'] != 'H']
ligand_noH.tail()


In [ ]:
ligand_coord = ligand_noH[["x", "y", "z"]].values

In [ ]:
distance_matrix_atm = sp.distance_matrix(ligand_coord, alpha_coord, p=2, threshold=1000000)

In [ ]:
indexArr_atm = np.unique(np.argwhere(distance_matrix_atm < 11)[:,1])

In [ ]:
indexArr_atm

In [ ]:
res_id = np.unique(alpha_carbon.loc[alpha_carbon.index.isin(indexArr_atm)]['residue_number'])

In [ ]:
res_id

In [ ]:
res_id.shape


In [ ]:
protein.df['ATOM'] = protein.df['ATOM'].loc[protein.df['ATOM']['residue_number'].isin(res_id)]

In [ ]:
protein.df['ATOM']=  protein.df['ATOM'][protein.df['ATOM']['line_idx'] > ter_pos[0]]
protein.df['ATOM']

In [ ]:
np.sort(np.unique(protein.df['ATOM']['residue_number']))

In [ ]:
np.sort(np.unique(protein_pocket.df['ATOM']['residue_number']))

In [ ]:
ligand_alpha = ligand.df[ligand.df['atom_name'] == 'CA']

In [ ]:
ligand_co = ligand_alpha[["x", "y", "z"]].values

In [ ]:
distance_matrix_atm = sp.distance_matrix(ligand_co, alpha_coord, p=2, threshold=1000000)

In [ ]:
indexArr_atm = np.unique(np.argwhere(distance_matrix_atm < 10)[:,1])

In [ ]:
indexArr_atm